# Model evaluation using test data set and all models

In [9]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.metrics import f1_score, jaccard_similarity_score

%matplotlib inline
Feature=pd.read_csv('Feature')
df = pd.read_csv('loan_train.csv')
X=np.load('X.npy')
y = df['loan_status'].values #labels
#X= preprocessing.StandardScaler().fit(X).transform(X)

from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore") 

In [10]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

--2019-11-16 14:41:16--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3642 (3.6K) [text/csv]
Saving to: ‘loan_test.csv’

loan_test.csv       100%[===================>]   3.56K  --.-KB/s    in 0s      

2019-11-16 14:41:16 (32.8 MB/s) - ‘loan_test.csv’ saved [3642/3642]



# Quick Preprocessing

In [11]:
from sklearn import preprocessing
test_df = pd.read_csv('loan_test.csv')


test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
Feature = test_df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(test_df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Xf = Feature

yf = test_df['loan_status'].values

Xf= preprocessing.StandardScaler().fit(Xf).transform(Xf.astype(float))
X_trainKNN=pd.read_csv('Xtrain_KNN')
y_trainKNN=pd.read_csv('ytrain_KNN')
X_trainset_tree=pd.read_csv('Xtrain_tree')
y_trainset_tree=pd.read_csv('ytrain_tree')
X_train_svm=pd.read_csv('Xtrain_svm')
y_train_svm=pd.read_csv('ytrain_svm')
X_train_LR=pd.read_csv('Xtrain_LR')
y_train_LR=pd.read_csv('ytrain_LR')

# KNN

In [12]:
k=9
neigh = KNeighborsClassifier(n_neighbors = 9).fit(X_trainKNN.values,y_trainKNN.values)

yhat_K = neigh.predict(Xf)
k1=jaccard_similarity_score(yf,yhat_K)
k2=f1_score(yf,yhat_K,average='weighted')

print('The F1 score is:',f1_score(yf, yhat_K, average='weighted')) 
print('The Jaccard Simmilarity Index is:',jaccard_similarity_score(yf, yhat_K))

The F1 score is: 0.6151515151515151
The Jaccard Simmilarity Index is: 0.6666666666666666


# Decision Tree

In [13]:
##Test Data-TREE
Tree = DecisionTreeClassifier(criterion="entropy", max_depth = 2)
Tree.fit(X_trainset_tree,y_trainset_tree)
predTree = Tree.predict(Xf)
t1=jaccard_similarity_score(yf,predTree)
t2=f1_score(yf,predTree,average='weighted')
print("DecisionTree's Jaccard Similarity Index: ", jaccard_similarity_score(yf, predTree))
print("DecisionTree's F1 Score is:",f1_score(yf,predTree,average='weighted'))

DecisionTree's Jaccard Similarity Index:  0.7407407407407407
DecisionTree's F1 Score is: 0.6304176516942475


# Support Vector Machine

In [14]:
##Test Data SVM
clf = svm.SVC(kernel='linear')
clf.fit(X_train_svm, y_train_svm) 

yhat_SVM = clf.predict(Xf)
s1=jaccard_similarity_score(yf,yhat_SVM)
s2=f1_score(yf,yhat_SVM,average='weighted')

print('The F1 score is:',f1_score(yf, yhat_SVM, average='weighted')) 
print('The Jaccard Simmilarity Index is:',jaccard_similarity_score(yf, yhat_SVM))

The F1 score is: 0.6304176516942475
The Jaccard Simmilarity Index is: 0.7407407407407407


# Logistic Regression

In [15]:
##Test Data Logistic
LR = LogisticRegression(C=0.1, solver='newton-cg').fit(X_train_LR,y_train_LR)
yhat_log = LR.predict(Xf)
yhat_prob = LR.predict_proba(Xf)
l1=jaccard_similarity_score(yf,yhat_log)
l2=f1_score(yf,yhat_log,average='weighted')
l3=log_loss(yf,yhat_prob)
print('The Jaccard Index is:',jaccard_similarity_score(yf, yhat_log))
print('The F1 score is:',f1_score(yf, yhat_log, average='weighted'))
print('The Log Loss is:',log_loss(yf, yhat_prob))

The Jaccard Index is: 0.7407407407407407
The F1 score is: 0.6304176516942475
The Log Loss is: 0.47101723936093937


In [16]:
data=[['KNN',k1,k2,'N/A'],['Decision Tree',t1,t2,'N/A'],['SVM',s1,s2,'N/A'],['LogisticRegression',l1,l2,l3]]
df=pd.DataFrame(data,columns=['Algorithm','Jaccard','F1-Score','LogLoss'])
df

,Algorithm,Jaccard,F1-Score,LogLoss
0,KNN,0.666667,0.615152,N/A
1,Decision Tree,0.740741,0.630418,N/A
2,SVM,0.740741,0.630418,N/A
3,LogisticRegression,0.740741,0.630418,0.471017


# In the future we will use grid search to optimize these parameters, as looping through is simply too messy